# GPU Version to run on MARCC

This notebook is assummed to be launched from the top level of the AutomaticLP github repo (where the notebook file is)

This also assummes that you've already cloned the tensorflow object detection project
https://github.com/tensorflow/models (since you need to do this to add the python path to  the bashrc file which has to be done before launching jupyter lab

Run with the LPproject kernel



**If you have not followed the instructions in install.readme quit this lab / notebook and follow thoes steps first !!!**

### Tests that Tensorflow GPU is installed & this notebook can see the GPU 

These don't need to be run everytime but are here as a sanity / debugging check

In [ ]:
# test that tensorflow can access a GPU
import tensorflow as tf
tf.test.is_built_with_cuda()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
# Make sure at least 1 GPU device is listed
print ('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

In [ ]:
# Expected Result is 0
!ml cuda/9.0
!export CUDA_VISIBLE_DEVICES=1
!echo ${CUDA_VISIBLE_DEVICES}

## Generating Datasets

The ScriptForBashFile generates both the training and test data and TFRecords from the image files


In [ ]:
%run ScriptForBashFile

## Train


In [ ]:
# This makes the output write to the file
# https://stackoverflow.com/questions/45200375/stdout-redirect-from-jupyter-notebook-is-landing-in-the-terminal
import sys
old_stdout = sys.stdout
sys.stdout = open('trainingLog.txt', 'w')

%cd models/research
%run object_detection/legacy/train.py --logtostderr --train_dir=../../trainingOutput/ --pipeline_config_path=../../configs/faster_rcnn_resnet101_coco.config 
%cd ../..

# Put the cell output back to normal
sys.stdout = old_stdout

In [ ]:
%matplotlib inline
%run viewTraining.py

## Evaluation



In [ ]:
# We want to run Evaluation on the latest checkpoint 
# this code extracts the step number of the most recent checkpoint saved

import os
from os import listdir
import re
import shutil

%cd trainingOutput
checkpointList = listdir()
%cd ..

numbers = []
for filename in checkpointList:
    match = re.search("^model.ckpt-", filename)
    if not match is None:
        filename = filename.split(".")
        filename = filename[1].split("-")
        numbers.append(filename[1])

lastCheckpoint = max(numbers)
print(lastCheckpoint)
CHECKPOINT_PREFIX = "../../trainingOutput/model.ckpt-"+str(lastCheckpoint)

# check to see if the infrence_graph folder exists - if it does ask if it should be deleted
# if the directory exists the TF code won't work
if(os.path.isdir("inference_graph")):
    choice = input("The Infrence Graph Directory already exists - to continue it needs to be deleted, is that OK (y/n)")
    if 'y' in choice.lower():
        shutil.rmtree('inference_graph')
    else:
        exit()
    
%cd models/research
%run object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path ./../configs/faster_rcnn_resnet101_coco.config --trained_checkpoint_prefix $CHECKPOINT_PREFIX --output_directory ../../inference_graph
%cd ../..


In [ ]:
%cd models/research
%run object_detection/inference/infer_detections.py --input_tfrecord_paths=../../FromScratch/TFRecordEval.tfrecord --inference_graph=../../inference_graph/frozen_inference_graph.pb --output_tfrecord_path=../../TFExamples.tfrecord
%cd ../..


In [ ]:
%cd tf_object_detection_cm
%run confusion_matrix.py --detections_record=../TFExamples.tfrecord --label_map=../FromScratch/data/label_map.pbtxt --output_path=confusion_matrix.csv
%cd ..

### Cells for augmentation test runs

In [ ]:
# Change the augmentation type as needed
%run augmentedTestDatasets.py -a GaussianNoise -n 1000

In [ ]:
%cd models/research
%run object_detection/inference/infer_detections.py --input_tfrecord_paths=../../FromScratch/TFRecordAugmented.tfrecord --inference_graph=../../inference_graph/frozen_inference_graph.pb --output_tfrecord_path=../../TFExamples.tfrecord
%cd ../..
